In [1]:
import time

now = 1598523184
local_tuple = time.localtime(now) # 컴퓨터의 시간대에 맞는 지역시간으로 변환 (한국 : KST = Asia/Seoul)
time_format = '%Y-%m-%d %H:%M:%S'
time_str = time.strftime(time_format, local_tuple)
print(time_str)

2020-08-27 19:13:04


In [3]:
# 반대로 변환
time_tuple = time.strptime(time_str, time_format)
utc_now = time.mktime(time_tuple)
print(utc_now)

1598523184.0


어떤 지역 시간대에 속한 시간을 어떻게 다른 지역 시간대로 변환할까?

time, localtime, strptime 함수가 반환하는 값을 직접 조적해 시간대를 변경하면

각 지역의 법에 따라 조작되므로 좋은 생각은 아니다..

세계 모든 주오ㅛ 도시의 출발/도축 시간을 변환하는건 복잡



In [5]:
# 시간대 변경을 자동으로 처리해주는 설정 파일을 여러 운영체제가 제공해준다
# 윈도우같은 플랫폼은 time이 제공하는 시간대 관련 기능 중 몇가지를 사용 할 수 없다

import os
if os.name == 'nt':
    print('이 예제는 윈도우에서 작동하지 않습니다')
    
else:
    parse_format = "%Y-%m-%d %H:%M:%S %Z" # %Z는 시간대를 뜻함
    depart_icn = '2020-08-27 19:13:04 KST'
    time_tuple = time.strptime(depart_icn, parse_format)
    time_str = time.strftime(time_format , time_tuple)
    print(time_str)

2020-08-27 19:13:04


사용자 컴퓨터의 운영체제나 언어 등의 locale 설정에따라 KST도 동작하지 않을 수 있다.

또한 strptime에 PDT인 미국 태평양 시간대를 사용하면 안됀다

In [6]:
arrival_sfo = '2020-08-28 04:13:04 PDT'
time_tuple = time.strptime(arrival_sfo, time_format)

# 플랫폼애 따라 다르게 작동한다. 그래서 python의 time 모듈은 신뢰할 수 없다.

ValueError: unconverted data remains:  PDT

In [7]:
from datetime import datetime, timezone

now = datetime(2020, 8, 27, 10, 13, 4) # 시간대 설정이 안된 시간을 만듦
now_utc = now.replace(tzinfo=timezone.utc) # 시간대를 UTC로 강제지정 
now_local = now_utc.astimezone() # UTC 시간대를 디폴트 시간대로 변환
print(now_local)


2020-08-27 19:13:04+09:00


In [8]:
time_str = '2020-08-27 19:13:04' # 지역 시간
now = datetime.strptime(time_str, time_format) # 시간대 설정이 안된 시간

time_tuple = now.timetuple() # 유닉스 시간 구조체로 변환
utc_now = time.mktime(time_tuple) # 구조체로 부터 유닉스 타임스탬프 생성
print(utc_now)

1598523184.0


datetime은 한 지역의 시간을 다른 지역 시간으로 바꾸는 신뢰할 수 있는 기능을 제공한다

하지만 datetime은 자신의 tzinfo 클래스와 이 클래스 안에 들어있는 메서드에 대해서만 시간대 관련 기능을 제공한다

파이썬 기본 설치에는 utc를 제외한 시간대 정의가 들어있지 않다

pytz 모듈은 모든 시간대의 완전한 데이터베이스가 들어있다

In [17]:
# 지역시간을 UTC로 먼저 바꿔야 한다
# datetime연산 offset 연산등 .. 마지막으로 UTC를 지역시간으로 바꿔라

# 샌프란시슼의 비행기 도착시간을 UTC datetime으로 변환

import pytz

arrival_sfo = '2023-12-21 17:15:14' # current local time
sfo_dt_naive = datetime.strptime(arrival_sfo, time_format) # 시간대가 설정되지 않은 시간
korea = pytz.timezone('Asia/Seoul') # 한국의 시간대
sfo_dt = korea.localize(sfo_dt_naive) # 시간대를 한국 시간대로 변경
utc_dt = pytz.utc.normalize(sfo_dt.astimezone(pytz.utc)) #UTC로 변경
print(utc_dt) # utc로 변경

2023-12-21 08:15:14+00:00


In [18]:
korea = pytz.timezone('Asia/Seoul')
korea_dt = korea.normalize(utc_dt.astimezone(korea)) # UTC datetime을 Korea로 변경
print(korea_dt)

2023-12-21 17:15:14+09:00


In [20]:
nepal = pytz.timezone('Asia/Katmandu')
nepal_dt = nepal.normalize(utc_dt.astimezone(nepal))
print(nepal_dt)

2023-12-21 14:00:14+05:45


In [23]:
sp = pytz.timezone('US/Pacific')
sp_dt = sp.normalize(utc_dt.astimezone(sp))
print(sp_dt)

2023-12-21 00:15:14-08:00
